In [ ]:
# pip install opencv-python

In [ ]:
# pip install tensorflow

In [ ]:
# pip install --upgrade tensorflowb

In [ ]:
pip install --upgrade jupyter

In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [2]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model, save_model

In [3]:
from tensorflow.keras.layers import Dropout, Activation
from tensorflow.keras.optimizers import Adam
from keras.regularizers import l2
import tensorflow as tf

In [4]:
tf.__version__

'2.10.1'

In [5]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
physical_devices = tf.config.list_physical_devices("GPU")
if physical_devices:
    # Get the number of available GPUs
    num_gpus = len(physical_devices)
    print(f"Number of available GPUs: {num_gpus}")
    # Print the name of each GPU
    for i in range(num_gpus):
        print(f"Using GPU {i}: {physical_devices[i].name}")
else:
    print("No GPU available, using CPU.")

Number of available GPUs: 1
Using GPU 0: /physical_device:GPU:0


In [7]:
epochs = 250
batch_size = 32

In [8]:
# Step 1: Read the text file
data = {}
with open('output.txt', 'r') as file:
    lines = file.readlines()
    for line in lines:
        if line.startswith('Image:'):
            image_name = line.split(': ')[1].strip()
#             print(image_name)
        elif line.startswith('Ductility Percentage:'):
            ductility = float(line.split(': ')[1].strip().strip('%'))
#             print(ductility)
        elif line.startswith('Brittle Percentage:'):
            brittleness = float(line.split(': ')[1].strip().strip('%'))
#             print(brittleness)
            data[image_name] = {'ductility': ductility, 'brittleness': brittleness}
        
# # Step 2: Load the images and preprocess

image_folder = r"D:\SEM Images\Filtered Data comp2"

images = []
labels = []
for image_name, info in data.items():
#     print(image_name)
    image_path = os.path.join(image_folder, image_name)
    image = cv2.imread(image_path)
    # Perform preprocessing if needed
    # e.g., resize, normalize, convert to grayscale, etc.
    image = cv2.resize(image, (228, 228))  # Resize the image
    image = image / 255.0  # Normalize pixel values
    images.append(image)
    labels.append((info['ductility'], info['brittleness']))



# # Step 3: Prepare data for CNN
# print(len(images))
X = np.array(images)
y = np.array(labels)

# # # Step 4: Split the data into training and validation sets
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

with tf.device('cpu:0'):
    dataset = tf.data.Dataset.from_tensor_slices((X, y)).batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

In [9]:
total_samples = len(X)
validation_samples = int(0.2*total_samples)
validation_batches = validation_samples // batch_size
validation_dataset = dataset.take(validation_batches)
training_dataset = dataset.skip(validation_batches)
steps_per_epoch = (total_samples - validation_samples)//batch_size
steps_per_epoch

447

In [ ]:
# # Step 5: Define and train the CNN model
model = Sequential()
model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(228, 228, 3), kernel_regularizer=l2(0.01)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.01)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.01)))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(2))  # One output for ductility and one for brittleness
model.summary()

model.compile(optimizer=Adam(learning_rate=0.0001), loss='mse', metrics=['accuracy'],)
# with tf.device('/gpu:0'):
# #     model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))
#     model.fit(training_dataset, validation_data = validation_dataset, epochs = epochs)

In [ ]:
model.save("model.h5")

In [10]:
model = load_model("model2.h5")

In [11]:
with tf.device('/gpu:0'):
#     model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))
    model.fit(training_dataset, validation_data = validation_dataset, epochs = epochs)

Epoch 1/250
448/448 [==============================] - 40s 70ms/step - loss: 0.5006 - accuracy: 0.9928 - val_loss: 1.9687 - val_accuracy: 0.9887
Epoch 2/250
448/448 [==============================] - 19s 41ms/step - loss: 0.4975 - accuracy: 0.9927 - val_loss: 2.0077 - val_accuracy: 0.9879
Epoch 3/250
448/448 [==============================] - 22s 48ms/step - loss: 0.4996 - accuracy: 0.9925 - val_loss: 2.0486 - val_accuracy: 0.9879
Epoch 4/250
448/448 [==============================] - 22s 46ms/step - loss: 0.5004 - accuracy: 0.9925 - val_loss: 2.0747 - val_accuracy: 0.9885
Epoch 5/250
448/448 [==============================] - 20s 42ms/step - loss: 0.5035 - accuracy: 0.9927 - val_loss: 2.1111 - val_accuracy: 0.9885
Epoch 6/250
448/448 [==============================] - 23s 49ms/step - loss: 0.5071 - accuracy: 0.9927 - val_loss: 2.1631 - val_accuracy: 0.9885
Epoch 7/250
448/448 [==============================] - 21s 45ms/step - loss: 0.5144 - accuracy: 0.9927 - val_loss: 2.2149 - val_ac

In [12]:
model.save("model3.h5")

In [ ]:
# # Step 6: Evaluate the model
loss, mae = model.evaluate(validation_dataset)
print("Validation Loss:", loss)
print("Validation Mean Absolute Error:", mae)

# # Step 7: Make predictions on new images
# def load_and_preprocess_image(image_path):
#     image = cv2.imread(image_path)
#     image = cv2.resize(image, (224, 224))
#     image = image / 255.0
#     image = np.expand_dims(image, axis=0)
#     return image

# new_image_path = "Part 1 - bending test on Motor carrios (Supplier-A).tiff"
# new_image = load_and_preprocess_image(new_image_path)

# # Make predictions
# predictions = model.predict(new_image)
# ductility_prediction = predictions[0][0]
# brittleness_prediction = predictions[0][1]

# # Print the predictions
# print(f'Predicted Ductility: {ductility_prediction:.2f}%')
# print(f'Predicted Brittleness: {brittleness_prediction:.2f}%')

In [ ]:
labels[0][0]

In [ ]:
# v1 = []
# v2 =[]
# for x in labels:
    
#     v1.append(x[0][0])
#     v2.append(x[0][1])
    
# v1 = v1/len(v1)
# v2 = v2/len(v2)
    
    